# Data Processing

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

c:\Users\dalzoj\Documents\Freelancer\TextClassificationProject


In [2]:
import src.utils.helper_functions as helpers
import src.utils.general_path as general_path


import re
import pandas as pd
from unidecode import unidecode
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
raw_analytic_df = pd.read_csv('../'+general_path.RAW_DATA_PATH + 'analytic_data.csv')
raw_analytic_df.head()

,label,text
0,SOL,SOLICITUD DE INTERVENCION CUMPLIMIENTO DE ORDE...
1,SOL,DERECHO DE PETICION - LA INSEMINACION ARTIFIC...
2,ASE,"MANIFIESTA EL SOLICITANTE, QUE ES VICTIMA DEL ..."
3,ASE,"MANIFIESTA EL SOLICITANTE, QUE ES VICTIMA DEL ..."
4,SOL,REPARTO ND 197/2022 - CASOS BARRERAS EMSSANAR ...


In [4]:
raw_correct_df = pd.read_excel('../'+general_path.RAW_DATA_PATH + 'correct_register.xlsx')
raw_correct_df.head()

c:\Users\dalzoj\Documents\Freelancer\TextClassificationProject\env\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Columna1,id_peticion\t,hechos_sample,Clasificación
0,0,2024036171,oeg-0314 notificaciondepacientesrepresadosenre...,NaN
1,1,2024039629,manifiesta que es cotizante de colpensiones y ...,NaN
2,2,2024026274,solicito respetuosamente en calidad de subsidi...,NaN
3,3,2024046117,habito en zona rural de una vereda denomidada ...,NaN
4,4,2024011185,patrullero adscritos a soledad de nombre braya...,NaN


In [5]:
correctly_processed_df = raw_correct_df.rename(
    columns = {
        'hechos_sample': 'text',
        'Clasificación': 'label'
        }
    )
correctly_processed_df = correctly_processed_df[['label','text']]
correctly_processed_df = correctly_processed_df.replace(
    to_replace = {
        'QUEJA': 'QJA',
        'SOLICITUD': 'SOL',
        'ASESORIA': 'ASE'
        }
    )
correctly_processed_df = helpers.df_preprocess(correctly_processed_df, True)
correctly_processed_df.head()

Elementos existentes: 3000
Elementos restantes: 1458


,label,text
0,QJA,presenta historia clinica de cehani fechada e...
1,QJA,"--usuaria cosmitet ltda refiere que, la doctor..."
2,QJA,usuario afiliado a la nueva eps regimen contri...
3,QJA,el pasado 22 de enero radique e informe antes ...
4,QJA,denuncia barrera administrativa eapb salud tot...


In [6]:
processed_analytical_df = helpers.df_preprocess(raw_analytic_df, True)
processed_analytical_df.head()

Elementos existentes: 37877
Elementos restantes: 28858


,label,text
0,SOL,SOLICITUD DE INTERVENCION CUMPLIMIENTO DE ORDE...
1,SOL,DERECHO DE PETICION - LA INSEMINACION ARTIFIC...
2,ASE,"MANIFIESTA EL SOLICITANTE, QUE ES VICTIMA DEL ..."
3,ASE,"MANIFIESTA EL SOLICITANTE, QUE ES VICTIMA DEL ..."
4,SOL,REPARTO ND 197/2022 - CASOS BARRERAS EMSSANAR ...


In [7]:
preprocess_df = pd.concat(
    [correctly_processed_df, processed_analytical_df]
    )
preprocess_df = helpers.df_preprocess(preprocess_df, True)
preprocess_df.head()

Elementos existentes: 30316
Elementos restantes: 30316


,label,text
0,QJA,presenta historia clinica de cehani fechada e...
1,QJA,"--usuaria cosmitet ltda refiere que, la doctor..."
2,QJA,usuario afiliado a la nueva eps regimen contri...
3,QJA,el pasado 22 de enero radique e informe antes ...
4,QJA,denuncia barrera administrativa eapb salud tot...


In [8]:
aditional_stopwords_list = general_path.EXTENDS_DATA_PATH + 'words_to_ignore.txt'
stop_words = stopwords.words("spanish")
stop_words.extend(aditional_stopwords_list)

In [9]:
# Expresión regular para texto
letters_pat = re.compile(r'[^a-z ]')
# Expresión regular para múltiples espacios
spaces_pat = re.compile(r' +')
# Expresión regular para puntos y comas
points_pat = re.compile(r'[,.-]')
# Crear el corrector ortográfico en español
spell = SpellChecker(language='es')

In [10]:
def text_processing(text):
    processed_text = unidecode(text.lower()) # Tildes
    processed_text = re.sub(letters_pat, " ", processed_text) # Eliminamos todo lo que no es letras y espacios
    processed_text = re.sub(points_pat, " ", processed_text) # Eliminamos puntuaciones
    processed_text = re.sub(spaces_pat, " ", processed_text) # Eliminamos mas de un espacio seguido
    processed_text = processed_text.strip() # Eliminamos los espacios al inicio y al final
    tokens = processed_text.split()
    tokens = [token for token in tokens if token not in stop_words]
    return " ".join(tokens)

In [11]:
def text_spellchecker_processing(text):
    processed_text = unidecode(text.lower()) # Tildes
    processed_text = re.sub(letters_pat, " ", processed_text) # Eliminamos todo lo que no es letras y espacios
    processed_text = re.sub(points_pat, " ", processed_text) # Eliminamos puntuaciones
    processed_text = re.sub(spaces_pat, " ", processed_text) # Eliminamos mas de un espacio seguido
    processed_text = processed_text.strip() # Eliminamos los espacios al inicio y al final
    tokens = processed_text.split()
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [spell.correction(token) if spell.correction(token) else token for token in tokens]
    preprocessing_text = unidecode(" ".join(tokens))
    return " ".join(tokens)

In [12]:
preprocess_df['text'] = preprocess_df['text'].apply(text_processing)
preprocess_df = helpers.df_preprocess(preprocess_df, True)
preprocess_df.head()

Elementos existentes: 30316
Elementos restantes: 28817


,label,text
0,QJA,presenta historia clinica cehani fechada indic...
1,QJA,usuaria cosmitet ltda refiere doctora aithza c...
2,QJA,usuario afiliado nueva eps regimen contributiv...
3,QJA,pasado enero radique informe respectivos entes...
4,QJA,denuncia barrera administrativa eapb salud tot...


In [13]:
preprocess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28817 entries, 0 to 28816
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   28817 non-null  object
 1   text    28817 non-null  object
dtypes: object(2)
memory usage: 450.4+ KB


In [14]:
preprocess_df.to_csv(
    '../'+general_path.PROCESSED_DATA_PATH + 'data.csv',
    index = False
    )

In [15]:
preprocess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28817 entries, 0 to 28816
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   28817 non-null  object
 1   text    28817 non-null  object
dtypes: object(2)
memory usage: 450.4+ KB


In [16]:
le = helpers.get_object('../'+general_path.OBJECTS_PATH,'label_encoder.pkl')
if le == None:
    le = LabelEncoder().fit(preprocess_df['label'])
    helpers.save_object('../'+general_path.OBJECTS_PATH, 'label_encoder.pkl', le)

Objeto cargado desde: ../src/objects/


In [17]:
# Transformación de las categorías
y_encoder = le.fit_transform(preprocess_df['label'])
preprocess_df['label'] = y_encoder

In [18]:
preprocess_df.to_csv(
    '../'+general_path.PROCESSED_DATA_PATH + 'processed_data.csv',
    index = False
    )